In [1]:
# GHOST - Advection Test (2D)

import sys  
import numpy as np
import modepy as mp
sys.path.insert(0, '../src')
from Mesh import Mesh2D
from Discretization import SpatialDiscretization
from Solver import Solver

# discretization degree
p = 4

# geometry mapping degree
p_geo = 1

# read in mesh in GMSH format
mesh = Mesh2D("test", "../mesh/square_mesh_L1_x_6y_6.msh")

# set up periodic boundary conditions
left = np.array([1.0,0.0,0.0]) 
right = np.array([1.0,0.0,1.0])
bottom = np.array([0.0,1.0,0.0])
top = np.array([0.0,1.0,1.0])
mesh.add_bc_on_hyperplanes([left,right,bottom,top],[1,2,3,4])
mesh.make_periodic((1,2),[1]) # left-right periodic (bcs parallel to axis 1)
mesh.make_periodic((3,4),[0]) # top-bottom periodic (axis 0)

#curvilinear transformation used in Del Rey Fernandez et al. (2017)
mesh.map_mesh(f_map=Mesh2D.grid_transformation(warp_factor=0.2), p_geo=p_geo)

# volume and facet quadrature degrees
tau = 2*p
mu = 2*p+1

theta = np.pi/4
a = np.sqrt(2)

# solver parameters
params = {"project_title": "advection_p4b0cpt1",
         "problem": "constant_advection",
         "initial_condition": "sine",
         "wavelength": np.ones(2),
         "wave_speed": a*np.array([np.sin(theta),np.cos(theta)]),
         "upwind_parameter": 0.0,
         "integration_type": "quadrature",
         "solution_degree": p,
         "volume_quadrature_degree": tau,
         "facet_quadrature_degree": mu,
         "solution_representation": "modal",
         "form": "weak",
         "correction": "c_+",
         "time_integrator": "rk44",
         "final_time": 1.0,
         "time_step_scale": 0.0025}

# set up solver
weak = Solver(params,mesh)
params_strong = params.copy()
params_strong["form"] = "strong"
strong = Solver(params_strong,mesh)

In [2]:
strong.run(write_interval=0.1)
strong.post_process(error_quadrature_degree=4*p)

dt =  2.777777777777778e-05
writing every  3600  time steps, total  36000
writing time step  3600 : t =  0.10000000000000467
max:  1.371780618288443
writing time step  7200 : t =  0.19999999999997284
max:  1.262807426280849
writing time step  10800 : t =  0.29999999999997845
max:  1.1879417799094496
writing time step  14400 : t =  0.40000000000003405
max:  1.2088553684682237
writing time step  18000 : t =  0.5000000000000896
max:  1.2693723829392052
writing time step  21600 : t =  0.5999999999999454
max:  1.382280256006858
writing time step  25200 : t =  0.6999999999998011
max:  1.269460596508793
writing time step  28800 : t =  0.7999999999996569
max:  1.1908972403950395
writing time step  32400 : t =  0.8999999999995126
max:  1.208689527983572
writing time step  36000 : t =  0.9999999999993684
max:  1.2735399881167797


In [3]:
weak.run(write_interval=0.1)
weak.post_process(error_quadrature_degree=4*p)

dt =  2.777777777777778e-05
writing every  3600  time steps, total  36000
writing time step  3600 : t =  0.10000000000000467
max:  1.3717806182884407
writing time step  7200 : t =  0.19999999999997284
max:  1.2628074262808469
writing time step  10800 : t =  0.29999999999997845
max:  1.1879417799094505
writing time step  14400 : t =  0.40000000000003405
max:  1.2088553684682228
writing time step  18000 : t =  0.5000000000000896
max:  1.2693723829392027
writing time step  21600 : t =  0.5999999999999454
max:  1.3822802560068541
writing time step  25200 : t =  0.6999999999998011
max:  1.269460596508792
writing time step  28800 : t =  0.7999999999996569
max:  1.190897240395036
writing time step  32400 : t =  0.8999999999995126
max:  1.208689527983568
writing time step  36000 : t =  0.9999999999993684
max:  1.2735399881167757


In [4]:
print("{:.3e}".format(strong.calculate_difference(weak)[0]), "&",
        "{:.3e}".format(strong.I_f[0] - strong.I_0[0]), "&",        
        "{:.3e}".format(weak.I_f[0] - weak.I_0[0]), "&",        
        "{:.3e}".format(strong.E_f[0] - strong.E_0[0]), "&",        
        "{:.3e}".format(weak.E_f[0] - weak.E_0[0]), "&",
        "{:.3e}".format(strong.calculate_error()[0]), "&",
        "{:.3e}".format(weak.calculate_error()[0]))

7.021e-15 & 9.751e-16 & 6.069e-16 & 7.216e-16 & -5.551e-17 & 2.409e-02 & 2.409e-02
